# Evaluate two CAD objects

In [96]:
import cadquery as cq
import numpy as np
from scipy.spatial import KDTree
import trimesh
from typing import Callable

## Setup

In [97]:
# These can be modified rather than hardcoding values for each dimension.
length = 80.0  # Length of the block
height = 60.0  # Height of the block
thickness = 10.0  # Thickness of the block
center_hole_dia = 22.0  # Diameter of center hole in block

box_cq = cq.Workplane("XY").box(length, height, thickness).faces(">Z")
box_cq.export("./box.stl")
holed_box_cq = box_cq.workplane().hole(center_hole_dia)
holed_box_cq.export("./holed_box.stl")

In [98]:
holed_box = trimesh.load_mesh("./holed_box.stl")
holed_box2 = trimesh.load_mesh("./holed_box.stl")
box = trimesh.load_mesh("./box.stl")
sphere = trimesh.creation.uv_sphere(radius=1.0)
cube = trimesh.creation.box(extents=(1, 1, 1))

## Evaluate

In [99]:
def eval_wrapper(error_value: float = 0, debug: bool = True):
    def decorator(function):
        def inner_wrapper(*args, **kwargs):
            try:
                return float(function(*args, **kwargs))
            except Exception as e:
                if debug:
                    raise e
                return error_value

        return inner_wrapper

    return decorator

### Intersection Over Union

In [100]:
@eval_wrapper()
def iou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    intersection_volume = 0
    for gt_mesh_i in obj1.split():
        for pred_mesh_i in obj2.split():
            intersection = gt_mesh_i.intersection(pred_mesh_i)
            volume = intersection.volume if intersection is not None else 0
            intersection_volume += volume

    gt_volume = sum(m.volume for m in obj1.split())
    pred_volume = sum(m.volume for m in obj2.split())
    union_volume = gt_volume + pred_volume - intersection_volume
    assert union_volume > 0
    return intersection_volume / union_volume

### Chamfer Distance

In [101]:
@eval_wrapper(10**9)
def chamfer_distance(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, num_samples: int = 5000
):
    points1, *_ = trimesh.sample.sample_surface(obj1, num_samples, seed=420)
    points2, *_ = trimesh.sample.sample_surface(obj2, num_samples, seed=420)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    return np.mean(np.square(gt_distance)) + np.mean(np.square(pred_distance))

    prox_query2 = trimesh.proximity.ProximityQuery(obj1)
    prox_query1 = trimesh.proximity.ProximityQuery(obj2)

    dist_1_to_2 = prox_query2.signed_distance(points1)
    dist_2_to_1 = prox_query1.signed_distance(points2)
    chamfer_dist = np.mean(dist_1_to_2**2) + np.mean(dist_2_to_1**2)

    return chamfer_dist

In [102]:
@eval_wrapper(10**9)
def chamfer_distance_vertices(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, max_points: int = 5000
):
    points1 = obj1.vertices
    points2 = obj2.vertices
    if len(points1) > max_points:
        np.random.seed(420)
        points1 = np.random.choice(points1, size=max_points, replace=False)

    if len(points2) > max_points:
        np.random.seed(420)
        points2 = np.random.choice(points2, size=max_points, replace=False)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    return np.mean(np.square(gt_distance)) + np.mean(np.square(pred_distance))

    # prox_query2 = trimesh.proximity.ProximityQuery(obj1)
    # prox_query1 = trimesh.proximity.ProximityQuery(obj2)

    # dist_1_to_2 = prox_query2.signed_distance(points1)
    # dist_2_to_1 = prox_query1.signed_distance(points2)
    # chamfer_dist = np.mean(dist_1_to_2**2) + np.mean(dist_2_to_1**2)

    # return chamfer_dist

### Combine

In [103]:
metrics_dict: dict[str, Callable[[trimesh.Trimesh, trimesh.Trimesh], float]] = {
    "iou": iou,
    "cd": chamfer_distance,
    "cdv": chamfer_distance_vertices,
}


def evaluate(
    obj1: trimesh.Trimesh,
    obj2: trimesh.Trimesh,
    metrics_dict: dict[
        str, Callable[[trimesh.Trimesh, trimesh.Trimesh], float]
    ] = metrics_dict,
) -> dict[str, float]:
    return {name: metric_fn(obj1, obj2) for name, metric_fn in metrics_dict.items()}

## Experiments

In [104]:
evaluate(holed_box, holed_box2)

{'iou': 1.0, 'cd': 0.0, 'cdv': 0.0}

In [105]:
evaluate(holed_box, box)

{'iou': 0.0, 'cd': 47.93233732139948, 'cdv': 1640.1535042437163}

In [106]:
evaluate(cube, sphere)

{'iou': 0.23973133347349573,
 'cd': 0.26070321919618245,
 'cdv': 0.3090600467025104}